# Introduction
## Why did we do this?
Many of the IQMs calculated are "no-reference" metrics. "A no-reference IQM is a measurement of some aspect of the actual image which cannot be compared to a reference value for the metric since there is no ground-truth about what this number should be."

Our goal was provide context for the image quality metrics (IQMs) shown in the MRIQC group reports, by showing the distribution of IQMs for your data plotted relative to a larger set of anonymized IQMs pulled from the web API.

## What MRIQCeption does
mriqception takes user IQMs from MRIQC and plots them relative to IQMs pulled from the 200k+ images in MRIQC web API (we're going to call those "normative" IQMs). The user has the option to filter their API query by relevant acquisition parameters, such as TR/TE.

mriqception also features a brief description of the IQM, shown as a tooltip when you mouseover the name of the IQM. We have tried to make these descriptions as user-friendly as possible.

In [1]:
# Import functions #
import argparse,datetime,os,sys,time

try:
    import plotly.graph_objects as go
except:
    go = None  

if go is None:
    print("plotly is not installed")

import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
#define widgets
modality_widget=widgets.RadioButtons(
    options=['bold', 'structural'],
    description='Modality:',
    disabled=False
)

TR_min=widgets.FloatSlider(
    min=1.5,
    max=5,
    step=0.1,
    description='TR min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)

In [7]:
TR_max=widgets.FloatSlider(
    min=1.5,
    max=4,
    step=0.1,
    description='TR max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)



TE_min=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


In [ ]:
TE_max=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


select_parameters=widgets.SelectMultiple(
    options=['TR_min', 'TR_max', 'TE_min', 'TE_max'],
    description='Parameters',
    disabled=False)

In [9]:
display(modality_widget)

Dropdown(description='Modality:', options=('bold', 'T1w', 'T2w'), value='bold')

In [10]:
modal=modality_widget.value
display(select_parameters)
"If empty, no parameter restrictions will applied"

SelectMultiple(description='Parameters', options=('TR', 'FD', 'SNR'), value=())

SelectMultiple(description='IQM to plot', options=('fwhm_avg', 'fd_mean', 'aor', 'aqi', 'efc'), value=())

In [12]:
for i in select_parameters.value:
        display(eval(i))

['TR > 1.25']


In [ ]:
to_filter=select_parameters.value
filter_list=[]
filter_dict={'TR_min': "TR >= {}".format(TR_min.value),
             'TR_max': "TR < {}".format(TR_max.value),
             'TE_min': "TE >= {}".format(TE_min.value),
             'TE_max': "TE < {}".format(TE_max.value)
            }

for item in to_filter:
    add_item=filter_dict.get(item)
    filter_list.append(add_item)
    
print(filter_list)
#filter_list= ['TR > 2.2', 'TR < 3']

In [13]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd

# path to input of local data from MRIQC on your own dataset 
group_file = os.path.join(here,'test_data', 'group_bold.tsv')

# scan type to query the API for [bold, T1w, T2w]
modality = 'bold'

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   SNR, TSNR, DVAR, FD,
   FWHM, Tesla, gsr_x, gsr_y, TE, TR
   NOTE: Only working as *and* right now!
"""


# IQM variables to visualize
#IQM_to_plot = ['aor','aqi','dummy_trs','dvars_nstd','dvars_std','dvars_vstd',
                    #'efc','fber','fd_mean','fd_num','fd_perc','fwhm_avg','fwhm_x','fwhm_y',
                    #'fwhm_z','gcor','gsr_x','gsr_y','snr','summary_bg_k','summary_bg_mad',
                   # 'summary_bg_mean','summary_bg_median','summary_bg_n','summary_bg_p05',
                   # 'summary_bg_p95','summary_bg_stdv','summary_fg_k','summary_fg_mad',
                   # 'summary_fg_mean','summary_fg_median','summary_fg_n','summary_fg_p05',
                   # 'summary_fg_p95','summary_fg_stdv','tsnr']

'Current possible filters:\n   SNR, TSNR, DVAR, FD,\n   FWHM, Tesla, gsr_x, gsr_y, TE, TR\n   NOTE: Only working as *and* right now!\n'

In [14]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape

(18, 45)

In [15]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
if not filter_list == []:
    filtered_apidf = filterIQM(apidf,filter_list)
else:
    filtered_apidf = apidf

# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)
#print(list(filtered_apidf))

(1000, 77)
(1000, 77)


In [16]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf.copy(), filtered_apidf.copy())
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape


(1018, 78)

In [17]:
# Visualization # 

v = make_vio_plot(vis_ready_df,IQM_to_plot,"",outliers=True)

widgets.VBox([v[0],v[1]])


Loading in dataframe...
Loading variables: aor
Loading in data descriptors...


    'data': [{'box'…